In [15]:
from __future__ import print_function, division

import plotly.graph_objects as go
import numpy as np
import scipy.spatial.distance
import math
import random
#from source import utils

import os
import glob
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.utils.data.dataset import random_split

In [16]:
import random

def read_pts(file):
    verts = np.genfromtxt(file)
    #return utils.cent_norm(verts)
    return verts

def read_seg(file):
    verts = np.genfromtxt(file, dtype= (int))
    return verts

def sample_2000(pts, pts_cat):    
    res1 = np.concatenate((pts,np.reshape(pts_cat, (pts_cat.shape[0], 1))), axis= 1)
    res = np.asarray(random.choices(res1, weights=None, cum_weights=None, k=2000))
    images = res[:, 0:3]
    categories = res[:, 3]
    categories-=np.ones(categories.shape)
    return images, categories

def read_data(file):
    data = np.genfromtxt(file).astype(np.float64)
    img = data[:, 0:3]
    seg = data[:, -1].astype(np.int64)
    return img, seg

def Normalize(pointcloud):
    norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0)
    norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

    return  norm_pointcloud

class RandRotation_z(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        theta = random.random() * 2. * math.pi
        rot_matrix = np.array([[ math.cos(theta), -math.sin(theta),    0],
                               [ math.sin(theta),  math.cos(theta),    0],
                               [0,                             0,      1]])
        
        rot_pointcloud = rot_matrix.dot(pointcloud.T).T
        return  rot_pointcloud
    
class Random_Noise(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        noise = np.random.normal(0, 0.02, (pointcloud.shape))
    
        noisy_pointcloud = pointcloud + noise
        return  noisy_pointcloud
    
    
def rotation_z(pointcloud, theta):
    rot_matrix = np.array([[ math.cos(theta), -math.sin(theta), 0],
                               [ math.sin(theta),  math.cos(theta), 0],
                               [0, 0, 1]])
        
    rot_pointcloud = rot_matrix.dot(pointcloud.T).T
    return  rot_pointcloud

def random_noise(pointcloud):
    noise = np.random.normal(0, 0.02, (pointcloud.shape))
    
    noisy_pointcloud = pointcloud + noise
    return  noisy_pointcloud

In [17]:
root_dir = '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet'
files = []
valid = False

In [18]:
newdir = root_dir + '/datasets/head_data/train/'

In [19]:
for file in os.listdir(newdir):
    o = {}
    o['path'] = newdir + file
    files.append(o)

In [20]:
print(files)

[{'path': '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet/datasets/head_data/train/5.txt'}, {'path': '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet/datasets/head_data/train/2.txt'}, {'path': '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet/datasets/head_data/train/1.txt'}, {'path': '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet/datasets/head_data/train/6.txt'}, {'path': '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet/datasets/head_data/train/3.txt'}, {'path': '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet/datasets/head_data/train/7.txt'}, {'path': '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet/datasets/head_data/train/4.txt'}]


In [21]:
idx = 0

path = files[idx]['path']

with open(path, 'r') as f:
    image1, category1 = read_data(f)

In [22]:
print(image1)

[[-9.81000e-03  0.00000e+00  1.44000e-04]
 [-1.62160e-02  0.00000e+00  1.44000e-04]
 [-2.90280e-02 -4.29000e-04  2.72000e-04]
 ...
 [-1.03814e-01  2.17127e-01  1.77469e-01]
 [-9.80260e-02  2.17264e-01  1.78167e-01]
 [-9.45360e-02  2.17177e-01  1.75970e-01]]


In [23]:
print(category1)

[0 0 0 ... 1 1 1]


In [24]:
print(image1.dtype, category1.dtype)

float64 int64


In [25]:
choice = np.random.choice(len(category1), 2000, replace=True)

image2 = image1[choice, :]
category2 = category1[choice]

In [27]:
print(len(image2))

2000


In [28]:
print(image2)

[[-8.26600e-02  1.80292e-01  1.82550e-01]
 [-1.16793e-01  1.76669e-01  1.84939e-01]
 [-1.25800e-01  1.79730e-01  1.74277e-01]
 ...
 [-1.18115e-01  1.64011e-01  1.57000e-04]
 [-5.98790e-02  2.26690e-02  1.07439e-01]
 [-1.84860e-01  1.70200e-02  1.16408e-01]]


In [29]:
print(category2)

[0 0 0 ... 0 0 0]


In [30]:
image2 = Normalize(image2)

In [31]:
print(image2)

[[ 0.11116547  0.19294254  0.47664963]
 [-0.06354444  0.1743982   0.48887774]
 [-0.1096468   0.19006594  0.43430424]
 ...
 [-0.0703111   0.10960817 -0.45692974]
 [ 0.22777012 -0.61385148  0.09219365]
 [-0.41194567 -0.64276591  0.13810151]]


In [11]:
#image2, category2 = sample_2000(image1, category1)

In [12]:
print(len(image1), len(image2))

9604 2000


In [13]:
print(image2)

[[-0.166637  0.168667  0.1531  ]
 [-0.168579  0.23178   0.140632]
 [-0.02577   0.180386  0.08828 ]
 ...
 [-0.078934  0.185817  0.174333]
 [-0.058219  0.104055  0.10099 ]
 [-0.060466  0.245283  0.162692]]


In [14]:
print(category2)

[-1. -1. -1. ... -1. -1. -1.]


In [14]:
image2 = Normalize(image2)

In [16]:
print(image2)

[[ 0.51744725 -0.42873993  0.0099299 ]
 [ 0.4096007  -0.40612538 -0.14602598]
 [-0.02920494 -0.63245912  0.08406328]
 ...
 [ 0.16526285 -0.72172653  0.14078283]
 [ 0.35254528 -0.02737245  0.01112068]
 [-0.36037267  0.21931641  0.27673472]]


In [19]:
theta = random.random()*360
image2 = rotation_z(random_noise(image2), theta)

In [20]:
print(image2)

[[ 0.56764485 -0.4147299   0.00469288]
 [ 0.46864284 -0.36729814 -0.15594394]
 [ 0.05250278 -0.63332298  0.06444678]
 ...
 [ 0.25924973 -0.64278727  0.09180772]
 [ 0.35572691  0.00112504 -0.03036878]
 [-0.40465991  0.18754191  0.28532905]]
